In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [2]:
# Set the computational backend
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using: {device}')

# Some helper functions for loading and saving trained models
def load_model(path, model, optimizer, scheduler=None):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler is None: return model, optimizer
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return model, optimizer, scheduler

def save_model(path, model, optimizer, scheduler=None):
    data = {'model_state_dict': model.state_dict()}
    if optimizer is not None: data['optimizer_state_dict'] = optimizer.state_dict()
    if scheduler is not None: data['scheduler_state_dict'] = scheduler.state_dict()
    torch.save(data, path)

# Some helper functions for retrieving CSV data (mesh and cfd results)
def get_csv_data(path, category, batch_size):
    if category == 'mesh':
        x, y, z = np.loadtxt(path, delimiter=',', unpack=True)
        x = torch.tensor(x, dtype=torch.float, requires_grad=True, device=device).reshape(-1,1)
        y = torch.tensor(y, dtype=torch.float, requires_grad=True, device=device).reshape(-1,1)
        z = torch.tensor(z, dtype=torch.float, requires_grad=True, device=device).reshape(-1,1)
        return DataLoader(TensorDataset(x, y, z), batch_size=batch_size, shuffle=True)
    elif category == 'cfd':
        x, y, z, p, u, v, w = np.loadtxt(path, delimiter=',', unpack=True)
        x = torch.tensor(x, dtype=torch.float, device=device).reshape(-1,1)
        y = torch.tensor(y, dtype=torch.float, device=device).reshape(-1,1)
        z = torch.tensor(z, dtype=torch.float, device=device).reshape(-1,1)
        u = torch.tensor(u, dtype=torch.float, device=device).reshape(-1,1)
        v = torch.tensor(v, dtype=torch.float, device=device).reshape(-1,1)
        w = torch.tensor(w, dtype=torch.float, device=device).reshape(-1,1)
        p = torch.tensor(p, dtype=torch.float, device=device).reshape(-1,1)
        return DataLoader(TensorDataset(x, y, z, u, v, w, p), batch_size=batch_size, shuffle=True)

def get_predictions(path, net_u, net_v, net_w, net_p):
    x, y, z = np.loadtxt(path, delimiter=',', unpack=True)

    res = np.zeros((len(x), 8))
    res[:, 0] = x
    res[:, 1] = y
    res[:, 2] = z

    x = torch.tensor(x, dtype=torch.float, requires_grad=True, device=device).reshape(-1,1)
    y = torch.tensor(y, dtype=torch.float, requires_grad=True, device=device).reshape(-1,1)
    z = torch.tensor(z, dtype=torch.float, requires_grad=True, device=device).reshape(-1,1)

    # pred_u = net_u(x,y,z).cpu().detach().numpy()
    # pred_v = net_v(x,y,z).cpu().detach().numpy()
    # pred_w = net_w(x,y,z).cpu().detach().numpy()
    # pred_p = net_p(x,y,z).cpu().detach().numpy()

    res[:, 3] = net_p(x,y,z).cpu().detach().numpy()[:,0]
    # res[:, 4]
    res[:, 5] = net_u(x,y,z).cpu().detach().numpy()[:,0]
    res[:, 6] = net_v(x,y,z).cpu().detach().numpy()[:,0]
    res[:, 7] = net_w(x,y,z).cpu().detach().numpy()[:,0]

    # return pred_u, pred_v, pred_w, pred_p
    return res

Using: cuda


In [3]:
# The neural network. Represents a function for \R^3 to \R
class NSNeuralNet(nn.Module):
    def __init__(self, width=256):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(3,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,width),
            nn.SiLU(),
            nn.Linear(width,1)
        )

    def forward(self, x, y, z):
        return self.main(torch.cat([x, y, z], axis=1))

In [4]:
# Setup the neural network, the optimizer, the learning rate scheduler, ...

model_u = NSNeuralNet().to(device)
model_v = NSNeuralNet().to(device)
model_w = NSNeuralNet().to(device)
model_p = NSNeuralNet().to(device)

model_u.apply(lambda m: nn.init.kaiming_normal_(m.weight) if type(m) == nn.Linear else None);
model_v.apply(lambda m: nn.init.kaiming_normal_(m.weight) if type(m) == nn.Linear else None);
model_w.apply(lambda m: nn.init.kaiming_normal_(m.weight) if type(m) == nn.Linear else None);
model_p.apply(lambda m: nn.init.kaiming_normal_(m.weight) if type(m) == nn.Linear else None);

In [5]:
optimizer_u = optim.Adam(model_u.parameters(), lr=0.1*1e-3)
optimizer_v = optim.Adam(model_v.parameters(), lr=0.1*1e-3)
optimizer_w = optim.Adam(model_w.parameters(), lr=0.1*1e-3)
optimizer_p = optim.Adam(model_p.parameters(), lr=0.1*1e-3)

In [6]:
# Load a pretrained model

model_u, optimizer_u = load_model('ns_34_u.pt', model_u, optimizer_u)
model_v, optimizer_v = load_model('ns_34_v.pt', model_v, optimizer_v)
model_w, optimizer_w = load_model('ns_34_w.pt', model_w, optimizer_w)
model_p, optimizer_p = load_model('ns_34_p.pt', model_p, optimizer_p)

model_u.train();
model_v.train();
model_w.train();
model_p.train();

FileNotFoundError: [Errno 2] No such file or directory: 'ns_34_u.pt'

In [7]:
# Domain of the PDE (the mesh stuff)
file_mesh = 'mesh_small_case_1.csv'
file_cfd = 'cfd_case_1_sample_1.csv'
batch_mesh = 5000
batch_cfd = 2500

mesh_ds = get_csv_data(file_mesh, 'mesh', batch_mesh)
cfd_ds = get_csv_data(file_cfd, 'cfd', batch_cfd)

FileNotFoundError: mesh_small_case_1.csv not found.

In [ ]:
loss_func = nn.MSELoss()
lambda_data = 10.0

rho = 1050 # density
mu = 0.0035 # viscosity

# Define the loss components due to the NS equations
def loss_physics(net_u, net_v, net_w, net_p, x, y, z):
    u = net_u(x, y, z)
    v = net_v(x, y, z)
    w = net_w(x, y, z)
    p = net_p(x, y, z)

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    p_z = torch.autograd.grad(p, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    w_x = torch.autograd.grad(w, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]

    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    w_y = torch.autograd.grad(w, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]

    u_z = torch.autograd.grad(u, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]
    v_z = torch.autograd.grad(v, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]
    w_z = torch.autograd.grad(w, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]

    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    w_xx = torch.autograd.grad(w_x, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]

    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    w_yy = torch.autograd.grad(w_y, y, grad_outputs=torch.ones_like(y), create_graph=True)[0]

    u_zz = torch.autograd.grad(u_z, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]
    v_zz = torch.autograd.grad(v_z, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]
    w_zz = torch.autograd.grad(w_z, z, grad_outputs=torch.ones_like(z), create_graph=True)[0]

    eqn_x = p_x + rho * (u*u_x + v*u_y + w*u_z) - mu * (u_xx + u_yy + u_zz)
    eqn_y = p_y + rho * (u*v_x + v*v_y + w*v_z) - mu * (v_xx + v_yy + v_zz)
    eqn_z = p_z + rho * (u*w_x + v*w_y + w*w_z) - mu * (w_xx + w_yy + w_zz)
    eqn_c = u_x + v_y + w_z

    eqn = eqn_x + eqn_y + eqn_z + eqn_c
    return loss_func(eqn, torch.zeros_like(eqn))

# Define the loss due to data fitting (regularisation)
def loss_data(net_u, net_v, net_w, net_p, x, y, z, u, v, w, p):
    cpt_u = loss_func(net_u(x, y, z), u)
    cpt_v = loss_func(net_v(x, y, z), v)
    cpt_w = loss_func(net_w(x, y, z), w)
    cpt_p = loss_func(net_p(x, y, z), p)
    return cpt_u + cpt_v + cpt_w + cpt_p

In [ ]:
# Training parameters
epochs = 200 # ideally around 10_000

print('P = Loss due to physics (NS equations)')
print('D = Loss due to data fitting')
print('T = Total loss')

def display_state(epochs_done, epochs_total, loss_phys, loss_data, loss_total):
    with torch.autograd.no_grad():
        msg = f'[{epochs_done:>8} / {epochs_total:<8}]'
        msg += f'    P( {loss_phys:>11.7f} )'
        msg += f'    D( {loss_data:>11.7f} )'
        msg += f'    T( {loss_total:>11.7f} )'
        print(msg)

for epoch in range(1,epochs+1):
    for xm, ym, zm in mesh_ds:
        for xd, yd, zd, ud, vd, wd, pd in cfd_ds:
            l_phys = loss_physics(model_u, model_v, model_w, model_p, xm, ym, zm)
            l_data = loss_data(model_u, model_v, model_w, model_p, xd, yd, zd, ud, vd, wd, pd)
            l_total = l_phys + lambda_data * l_data

            optimizer_u.zero_grad()
            optimizer_v.zero_grad()
            optimizer_w.zero_grad()
            optimizer_p.zero_grad()

            l_total.backward()

            optimizer_u.step()
            optimizer_v.step()
            optimizer_w.step()
            optimizer_p.step()

    if epoch % 5 == 0:
        display_state(epoch, epochs, l_phys, l_data, l_total)


P = Loss due to physics (NS equations)
D = Loss due to data fitting
T = Total loss


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[       5 / 200     ]    P(   0.0419895 )    D(   0.2522198 )    T(   2.5641873 )
[      10 / 200     ]    P(   0.0049560 )    D(   0.2415206 )    T(   2.4201622 )
[      15 / 200     ]    P(   0.0018223 )    D(   0.2392178 )    T(   2.3940001 )
[      20 / 200     ]    P(   0.0013844 )    D(   0.2388716 )    T(   2.3901002 )
[      25 / 200     ]    P(   0.0155211 )    D(   0.2390309 )    T(   2.4058297 )
[      30 / 200     ]    P(   0.0086552 )    D(   0.2404910 )    T(   2.4135656 )
[      35 / 200     ]    P(   0.0011863 )    D(   0.2374698 )    T(   2.3758841 )
[      40 / 200     ]    P(   0.0027596 )    D(   0.2380478 )    T(   2.3832374 )
[      45 / 200     ]    P(   0.0021021 )    D(   0.2372353 )    T(   2.3744547 )
[      50 / 200     ]    P(   0.0130470 )    D(   0.2378717 )    T(   2.3917639 )
[      55 / 200     ]    P(   0.0017009 )    D(   0.2363320 )    T(   2.3650205 )
[      60 / 200     ]    P(   0.0010208 )    D(   0.2357431 )    T(   2.3584516 )
[      65 / 200 

In [ ]:
# # Save the trained model

save_model('ns_34_u.pt', model_u, optimizer_u)
save_model('ns_34_v.pt', model_v, optimizer_v)
save_model('ns_34_w.pt', model_w, optimizer_w)
save_model('ns_34_p.pt', model_p, optimizer_p)

In [ ]:
# # Use the trained model to generate the velocity and pressure fields

dt = get_predictions('calc_mesh.csv', model_u, model_v, model_w, model_p)
np.savetxt('1_xz_full_pinn.csv', dt, delimiter=',', fmt='%.8e')

# # Make plots, or export to VTK etc